# Dune Toe and Dune Ridge Feature Extraction

*Disclaimer: This notebook is in draft form and may require adjustments to run smoothly. If you encounter any issues, please inform us so we can assist you.* 

Shoreline dune migration refers to the natural movement of sand dunes along coastal areas. This process is driven by various factors, primarily wind but also waves and tidal currents. The sand is transported from one place to another, causing the dunes to gradually shift in position and shape over time.

This migration is an important natural phenomenon because it affects coastal ecosystems, wildlife habitats, and even human infrastructure. The movement of dunes can provide natural coastal protection against storms and sea-level rise, but it can also pose challenges for coastal management, especially in areas where development is close to the shore.

Monitoring and managing dune migration are crucial for maintaining the balance between protecting natural landscapes and supporting human activities along coastlines. The following figures shows the shorelines dune migration (images from [Hardin et al. (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_4).

Based on [Hardin et al. (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_4)

![](../img/dunes_migration.png)

![](../img/dunes_migration_1.png)

***

## 1. Import Python Packages

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

***

## 2. Start GRASS Session

In [ ]:
gj.init("nags_head/PERMANENT")


In [ ]:
!g.region raster="JR_2014" -p

***

## 3. Foredune Ridge Line
* Purpose: Extract a dune ridge as a least cost path.

* Specify two points that were manually selected at

* opposite ends of dune ridge.

* Compute cost surface.

In [ ]:
# Compute cost surface.
!r.mapcalc expression='cost=exp(-2*JR_2014)' --o  

# Compute a cumulative cost surface.
!r.cost -k input=cost output=cumulative_cost start_coordinates='913859,250658' stop_coordinates='914305,249739' --o

# Calculate the least coast path.
!r.drain input=cumulative_cost output=JR_2014_duneRidge start_coordinates='914305,249739' --o
# Extract dune ridge.

!r.mapcalc \
expression='JR_2014_duneRidge=float\
(JR_2014_duneRidge)*JR_2014' --o


In [ ]:
!r.to.vect -s input=JR_2014_duneRidge output=JR_2014_duneRidge_vector type=line

***

## 4. Visualization

In [ ]:
!r.colors map=JR_2014 color=elevation
!r.colors map=JR_2014_duneRidge color=blue

fig = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
fig.add_raster("JR_2014")
fig.add_vector("JR_2014_duneRidge_vector")
fig.add_layer_control()
fig.show()

***

## 4. Foredune Toe Line
* Purpose: Compute dune toe line with elastic sheet

In [ ]:
!g.list type=vect

In [ ]:
!g.list type=rast

In [ ]:
#!v.build map=JR_2014 option=build

In [ ]:
# conditions for efficiency.
gs.run_command( 'v.patch', input='JR_2014_duneRidge_vector,JR_2014_08m', output='sheet_BC')

In [ ]:
gs.run_command( 'g.region', vect='sheet_BC' )

In [ ]:
gs.run_command( 'v.to.rast', input='JR_2014_08m', output='JR_2014_08m', use='val', value='0.8' )

In [ ]:
gs.run_command( 'r.patch', input='JR_2014_duneRidge,JR_2014_08m', output='sheet_BC', overwrite=True)

In [ ]:
gs.run_command( 'g.copy', rast='JR_2014,sheet' )

In [ ]:
iterations = 3000
for i in range(iterations):
    #print (i)
    gs.run_command( 'r.neighbors', flags='c', input='sheet', output='sheet', method='average', size=3, overwrite=True )
    gs.run_command( 'r.patch', input='sheet_BC,sheet', output='sheet', overwrite=True )
gs.run_command( 'r.colors', map='sheet', rast='JR_2014' )

In [ ]:

# Make small null buffer around dune ridge and

# shoreline to keep extracted toe between them.

gs.run_command( 'r.buffer', input='sheet_BC', output='sheet_BC_buff', dist=1 )
gs.run_command( 'r.mapcalc', expression='deviation_map=if(isnull(sheet_BC_buff), sheet-JR_2014, null())' )

# Again, use two manually selected points.

pt1 = '913878,250654'
pt2 = '914317,249759'

# Extract dune toe.
gs.run_command( 'r.mapcalc', expression='cost=exp(-5*deviation_map)' )
gs.run_command( 'r.mapcalc', expression='cost=exp(-5*deviation_map)' )
gs.run_command( 'r.drain', input='cumulative_cost', output='JR_2014_duneToe_new', start_coordinates=pt2 )
gs.run_command( 'r.mapcalc', expression='JR_2014_duneToe_new=float(JR_2014_duneToe_new)*JR_2014' )

In [ ]:
!g.list type=raster

***

## 5. Visualization

In [ ]:
#dune = gj.Map(use_region=True)
#dune.d_rast(map="JR_2014_duneToe_new") # d.rast map=naip_2020.1
#dune.show()

In [ ]:
#dunes = gj.Map(use_region="JR_2014_duneRidge_vector")
dunes = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
dunes.add_raster("JR_2014")
dunes.add_raster("JR_2014_duneToe_new")
dunes.add_vector("JR_2014_duneRidge_vector")
dunes.add_vector("JR_2014_shore")
dunes.add_layer_control()
dunes.show()

## Further Readings

[Hardin E., Kurum O., Mitasova H., Overton MF, 2012, Least cost path extraction of topographic features for storm impact scale mapping, Journal of Coastal Research 28(4), p. 970-978.](https://doi.org/10.2112/JCOASTRES-D-11-00126.1)